In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

# 计算最大回撤
def calculate_max_drawdown(data):
    max_dd = [0] * len(data)
    max_dd[0] = 0
    peak = data.iloc[0]

    for i in range(len(data)):
        if data.iloc[i] > peak:
            peak = data.iloc[i]
        else:
            drawdown = (peak - data.iloc[i]) / peak
            max_dd[i] = min(0,-max(max_dd[i], drawdown))

    return max_dd


# 计算策略评测指标
def performance_cal(daily_df,market_value,rf):
    column = ['trade_count','cost','t_pnl','h_pnl','net_pnl','cum_pnl','nv','daily_ret','max_dd']
    # 交易表现
    # trade = simple_strategy.get_trades(symbol) # 获取交易记录
    port_df = pd.DataFrame(index = daily_df.index,columns = column)
    # trade_count为交易次数非手数
    port_df[['trade_count','t_pnl','h_pnl','net_pnl','cum_pnl']] = daily_df[['trade_count','trading_pnl','holding_pnl','net_pnl','balance']]
    port_df['nv'] = port_df['cum_pnl'] / market_value
    port_df['daily_ret'] = port_df['nv'].pct_change()
    port_df['max_dd'] = calculate_max_drawdown(port_df['nv'])
    years = len(port_df) / 252 # 回测长度（以年为单位）
    total_return = port_df['nv'].iloc[-1] - 1 # 累计收益率
    annualized_return = (total_return + 1) ** (1 / years) - 1 if total_return > 0 else 0 # 年化收益率
    annualized_vol = port_df['daily_ret'].std() * np.sqrt(252) # 年化波动率
    sharpe_ratio = (annualized_return - rf) / annualized_vol  # 夏普比率
    max_dd = port_df['max_dd'].min() # 历史最大回撤，带负号，小数形式
    win_prob = (port_df['daily_ret'] > 0).sum() / len(port_df) # 日度胜率，非交易胜率
    trade_count_all,t_pnl,h_pnl = port_df[['trade_count','t_pnl','h_pnl']].sum() # 总交易次数，手续费，交易损益，持仓损益
    # 平均盈亏比（盈利的天数里收益率的平均值/亏损的天数里收益率的平均值）
    avg_pnl_ratio = port_df[port_df['daily_ret'] > 0]['daily_ret'].mean() / port_df[port_df['daily_ret'] < 0]['daily_ret'].abs().mean()
    rr_ratio = annualized_return / annualized_vol # 收益风险比
    calmar_ratio = 0 if annualized_return <= 0 else annualized_return / np.abs(max_dd) # 卡玛比率

    result_list = [annualized_return, annualized_vol, sharpe_ratio, max_dd, win_prob, trade_count_all, t_pnl, h_pnl, avg_pnl_ratio, rr_ratio, calmar_ratio]
    result_df = pd.DataFrame(result_list,index = ["年化收益率","年化波动率","夏普比率","最大回撤","胜率","交易次数","交易损益","持仓损益","平均盈亏比","收益风险比","卡玛比率"]).T
    return port_df,result_df

def ATR_signal_cal(df,N,k1,k2):
    # 计算指标
    ohlc_df = df[['Open','High','Low','Close']].copy()
    # 计算TR
    ohlc_df['H-L'] = ohlc_df['High'] - ohlc_df['Low']
    ohlc_df['abs(H-C_1)'] = (ohlc_df['High'] - ohlc_df['Close'].shift(1)).abs()
    ohlc_df['abs(L-C_1)'] = (ohlc_df['Low'] - ohlc_df['Close'].shift(1)).abs()
    ohlc_df['TR'] = ohlc_df[['H-L','abs(H-C_1)','abs(L-C_1)']].max(axis=1)
    # 计算ATR
    ohlc_df['ATR'] = ohlc_df['TR'].rolling(N).mean()
    print(ohlc_df.head(20))
    print(ohlc_df.tail(20))
    # 计算上下轨
    ohlc_df['Upline'] = ohlc_df['Open'] + k1 * ohlc_df['ATR']
    ohlc_df['Downline'] = ohlc_df['Open'] - k2 * ohlc_df['ATR']
    print(ohlc_df.head(20))
    print(ohlc_df.tail(20))
    # 生成突破信号
    ohlc_df['signal'] = 0
    ohlc_df.loc[ohlc_df[ohlc_df['Close'] > ohlc_df['Upline']].index,'signal'] = 1
    ohlc_df.loc[ohlc_df[ohlc_df['Close'] < ohlc_df['Downline']].index,'signal'] = -1
    print(ohlc_df.head(20))
    print(ohlc_df.tail(20))

    return ohlc_df[['signal']]

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

sp500 = yf.Ticker("^GSPC")

# Get data for a specific date range
data = sp500.history(start="2020-01-01", end="2024-01-03")

In [ ]:
N, k1, k2 = 7, 0.8, 0.8

signal = ATR_signal_cal(data,N,k1,k2)

                                  Open         High          Low        Close  \
Date                                                                            
2020-01-02 00:00:00-05:00  3244.669922  3258.139893  3235.530029  3257.850098   
2020-01-03 00:00:00-05:00  3226.360107  3246.149902  3222.340088  3234.850098   
2020-01-06 00:00:00-05:00  3217.550049  3246.840088  3214.639893  3246.280029   
2020-01-07 00:00:00-05:00  3241.860107  3244.909912  3232.429932  3237.179932   
2020-01-08 00:00:00-05:00  3238.590088  3267.070068  3236.669922  3253.050049   
2020-01-09 00:00:00-05:00  3266.030029  3275.580078  3263.669922  3274.699951   
2020-01-10 00:00:00-05:00  3281.810059  3282.989990  3260.860107  3265.350098   
2020-01-13 00:00:00-05:00  3271.129883  3288.129883  3268.429932  3288.129883   
2020-01-14 00:00:00-05:00  3285.350098  3294.250000  3277.189941  3283.149902   
2020-01-15 00:00:00-05:00  3282.270020  3298.659912  3280.689941  3289.290039   
2020-01-16 00:00:00-05:00  3

In [ ]:
signal.value_counts()

,count
signal,
0,824
-1,98
1,85
